In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import numpy as np
import tensorflow as tf
from tqdm import tqdm

In [3]:
#f = open("473.astar-s0.txt", 'r')
f = open('/content/drive/MyDrive/Colab Notebooks/dataset/473.astar-s0.txt', 'r')

In [4]:
load_address = list()

f.seek(0, 0)
while True:
  line = f.readline()
  if not line: break
  split_line = line.split(', ')
  load_address.append(split_line[2])

for i in range(len(load_address)):
  load_address[i] = int(load_address[i], 16)//64

print("\nload_address example: ")
print(load_address[:100])
print(len(load_address))


load_address example: 
[702778646977, 702778647164, 702778646976, 702778646921, 702778647176, 702778647287, 702778647353, 702778647368, 702778647452, 702778647286, 702778647484, 702778647560, 702778647644, 702778647688, 702778647352, 702778647755, 702778647819, 702778647901, 702778647933, 4363500793931, 702778647351, 702778647820, 702778647756, 4363500793932, 4363500794023, 4363500794087, 174026416743, 702778647350, 174026416807, 174026416838, 2627809326406, 2627809326492, 2627809326524, 2627809326534, 2627809326620, 702778647349, 2627809326662, 2627809326780, 2627809326844, 2627809326857, 2627809326889, 2627809326972, 2627809326988, 2627809327052, 2627809327116, 2627809327180, 2627809327277, 2627809327309, 2627809327341, 702778647348, 2627809327115, 2627809327366, 2627809327430, 2627809327532, 2627809327564, 2627809327596, 2627809327622, 2627809327686, 2627809327749, 702778647347, 2627809327813, 2627809327877, 2627809327979, 2627809328011, 2627809328043, 2627809327947, 2627809328069,

In [5]:
offset_dic = dict()
for i in range(len(load_address)):
  if i == 100000:
    print(offset_dic)
    break
  load = load_address[i] >> 10
  if load in offset_dic:
    offset_dic[load].append(load_address[i] % (1 << 10))
  else:
    offset_dic[load] = [load_address[i] % (1 << 10)]

{686307272: [449, 636, 448, 393, 648, 759, 825, 840, 924, 758, 956, 824, 823, 822, 821, 820, 819, 818, 817, 816, 815, 814, 813, 812, 811, 810, 809, 808, 807, 806, 805, 804, 803, 802, 801, 800, 799, 798, 797, 796, 795, 794, 793, 792, 791, 790, 789, 788, 787, 786, 785, 784, 783, 782, 781, 780, 779, 778, 777, 776, 775, 774, 773, 772, 771, 770, 769, 768, 916, 948, 760, 912, 944, 761, 762, 680, 763, 872, 932, 764, 765, 700, 701, 892, 893, 937, 905, 766, 767, 872, 932, 680, 681, 704, 705, 706, 682, 707, 874, 708, 874, 874, 933, 874, 709, 874, 901, 710, 711, 712, 713, 714, 715, 716, 717, 718, 949, 917, 719, 720, 721, 722, 723, 954, 953, 921, 922, 724, 725, 726, 898, 897, 908, 929, 861, 865, 930, 940, 673, 864, 669, 727, 672, 728, 729, 730, 691, 883, 731, 935, 882, 690, 903, 732, 733, 734, 735, 736, 737, 738, 914, 739, 946, 394, 395, 581, 577, 401, 397, 398, 396, 399, 730, 731, 732, 734, 735, 736, 737, 738, 750, 751, 753, 754, 755, 759, 760, 761, 762, 763, 764, 765, 766, 767, 704, 705, 706, 70

In [6]:
seq_length = 3
output_length = 3

In [7]:
offset_address = np.array(list(offset_dic.keys()))

print(offset_address)
print(offset_address.shape)
print(offset_address[1])

[ 686307272  686307273 4261231244  169947672 2566220045 2566220046
 2566220047 2566220048 2566220049 2566220050 2566220051 2566220052
 2566220053 2566220054 2566220055 2566220056 2566220057 2566220058
 2566220059 2566220060 2566220061 2329096219 2329096220  245499435
  245499436  245499437  245499438  245499439  245499440  245499441
  245499442  245499443 2712393725 2712393726 1819221580 3617481380
 3617481381 3617481382 3617481383 3617481384 3617481385 3617481386
 3617481387 3617481388 3617481389 3617481390 3617481391 3617481392
 3617481393 3617481394 3617481395 3617481396 3617481397 3617481398
 3617481399 3617481400 3617481401 3617481402 3617481403 3617481404
 3617481405 3617481406 3617481407 3617481408 3617481409 3617481410
 3617481411 3617481412 3892534853 3892534854 3892534855 1817407538
 1817407539 1817407540 2815462096 3996171156 1479124715 1479124716
 1479124717 1093563338 1093563339 1093563340 1093563341 1093563342
 1909847007 1909847008 1909847009 1909847010 1909847011 190984

In [8]:
def address_to_binary(a, l):
  binary_int = list()
  for i in range(l):
    binary_int.append(a%2)
    a = a//2
  return binary_int

In [9]:
weight_lean = 0.5
l = 33
scale = 0.1
address_binary_input = list()
address = list()
rnn_input = list()
multi_output = list()
categorical_multi_output = list()

for i in tqdm(range(len(offset_address))):
  offset_list = offset_dic[offset_address[i]]
  if len(offset_list) < seq_length:
    continue

  for j in [0] + list(range(1, len(offset_list) - seq_length - output_length + 1)):
    address_binary_input.append(address_to_binary(offset_address[i], l))
    rnn_input.append(offset_list[j:j+seq_length])
    if len(offset_list) < seq_length + output_length:
      multi_output = offset_list[j+seq_length:]
    else:
      multi_output = offset_list[j+seq_length:j+seq_length+output_length]

    tmp = [0 for k in range(1024)]
    for k in range(len(multi_output)):
      tmp[multi_output[k]] = round(1 - weight_lean * k/output_length,2)
    categorical_multi_output.append(tmp)

address_binary_input = np.array(address_binary_input)
rnn_input = np.array(rnn_input)
categorical_multi_output = np.array(categorical_multi_output)

print(f'{address_binary_input.shape}')
print(f'{rnn_input.shape}')
print(f'{categorical_multi_output.shape}')

100%|██████████| 355/355 [00:09<00:00, 37.69it/s]


(98231, 33)

(98231, 3)

(98231, 1024)



In [10]:
# shuffle_data
idx = np.arange(address_binary_input.shape[0])
np.random.shuffle(idx)

address_binary_input = address_binary_input[idx]
rnn_input = rnn_input[idx]
categorical_multi_output = categorical_multi_output[idx]